In [15]:
#import dependencies
import pandas
import numpy
import hvplot.pandas
#machine learning dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

#connection to db
from connect_sql_db import build_engine


#look for labelencoder

In [4]:
engine = build_engine(database_name="database1",host="35.225.193.21")

In [ ]:
#read in csvs
review_csv = pandas.read_csv("../data/yelp_dataset/yelp_csvs/yelp_review_raw.csv", low_memory=False)

In [5]:
#read data from sql 
business_table = pandas.read_sql("select * from business_table", con=engine)

In [6]:
business_table.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open
0,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,61820,40.110446,-88.233073,4.5,5,1
1,CsLQLiRoafpJPJSkNX2h5Q,Middle East Deli,4508 E Independence Blvd,Charlotte,NC,28205,35.194894,-80.767442,3.0,5,0
2,eBEfgOPG7pvFhb2wcG9I7w,Philthy Phillys,"15480 Bayview Avenue, unit D0110",Aurora,ON,L4G 7J1,44.010962,-79.448677,4.5,4,1
3,lu7vtrp_bE9PnxWfA8g4Pg,Banzai Sushi,300 John Street,Thornhill,ON,L3T 5W4,43.820492,-79.398466,4.5,7,1
4,9sRGfSVEfLhN_km60YruTA,Apadana Restaurant,13071 Yonge Street,Richmond Hill,ON,L4E 1A5,43.947011,-79.454862,3.0,3,1


In [7]:
#create a list of names where total count of reviews is above 50
above_50_reviews = business_table.groupby("name").sum()["review_count"][business_table.groupby("name").sum()["review_count"] > 50].index.tolist()

In [8]:
#column based on list
business_table["above_50"] =  business_table.name.apply(lambda x: True if x in above_50_reviews else False)

In [10]:
#new table where all restaurant reviews are greater than 50
new_business_table = business_table.loc[business_table.above_50 == True].drop(["above_50", "is_open"],axis=1)

In [11]:
new_business_table

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count
6,fnZrZlqW1Z8iWgTVDfv_MA,Carl's Jr,9595 W Tropicana Ave,Las Vegas,NV,89147,36.099738,-115.301568,2.5,15
10,LoRef3ChgZKbxUio-sHgQg,Amir,5252 Rue Jean Talon O,Montréal,QC,H4P 2A7,45.494870,-73.651904,3.0,18
13,tLpkSwdtqqoXwU0JAGnApw,Wendy's,4602 Northfield Road,Cleveland,OH,44128,41.434614,-81.527026,3.5,7
14,Sd75ucXKoZUM2BEfBHFUOg,China Gourmet,"3460 E Southern Ave, Ste 109",Mesa,AZ,85204,33.394863,-111.756036,3.0,13
16,-qjn24n8HYF6It9GQrQntw,Five Guys Burgers and Fries,2130 E Arbors Dr,Charlotte,NC,28262,35.338070,-80.757397,4.0,15
...,...,...,...,...,...,...,...,...,...,...
63432,YZeUH6zYS0dq5QHLYZhUnQ,Hooters,2820 N 75th Ave,Phoenix,AZ,85035,33.479139,-112.221142,2.0,106
63433,xVpE01l6ZXdEtVf5PkRpDg,Julep,829 E Washington Ave,Madison,WI,53703,43.081022,-89.374006,4.0,95
63434,BAVuLTDmpSzDCk37A5HjtQ,Bruegger's Bagels,27045 Lorain Rd,North Olmsted,OH,44070,41.416568,-81.921396,3.0,13
63435,hskVqZCPqy-omm9CHi44xQ,Domino's,"6420 Rea Rd, Suite B1",Charlotte,NC,28226,35.078538,-80.818358,2.0,16


In [12]:
y = new_business_table.stars
X = new_business_table.drop("stars", axis = 1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state =1)

In [ ]:
sm = SMOTE(random_state = 1)
X_resampled, y_resampled, = sm.fit_resample(X_train, y_train)

In [33]:
model = LogisticRegression(
    solver = "lbfgs",
    random_state = 1
)

model.fit(X_resampled, y_resampled)

A-Vm34siEd2XchdYsZbeqw    1
mT8Ej8sKFJoIa5G0Vl4AKA    1
TbHiV1t65v_Tyr0_ro1tbg    1
zWKpPyzm0W9N5OEPDuwgMA    1
i0bHCjfICwUTcZ1NXSFlvw    1
                         ..
xVgFFbACM8Qe1W6CIk48jg    1
_Ejjcg6V2sHb_W8CyIUtXg    1
XUlQ4BVoZH6rnAT40cYGOQ    1
GdnLAgRXt43eISMtJEOZRg    1
qKS7YV76te6olKnA_aMPTg    1
Name: business_id, Length: 34340, dtype: int64 



McDonald's                 854
Subway Restaurants         613
Burger King                336
Taco Bell                  334
Pizza Hut                  330
                          ... 
New City Restaurant          1
Wolfgang Puck Pizza Bar      1
Pampas Las Vegas             1
Hakkasan Restaurant          1
The Buffet at Bellagio       1
Name: name, Length: 15807, dtype: int64 



5757 Wayne Newton Blvd     30
4321 W Flamingo Rd         27
3600 S Las Vegas Blvd      22
3131 Las Vegas Blvd S      22
11011 W Charleston Blvd    20
                           ..
16433 West Bell Rd          1
4315 E Indian School Rd     1
647 E Mckellips 

In [ ]:
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [43]:
#collect all possible categories
categories = busi_csv.categories.tolist()

categories_lst = []
for i in categories:
    if type(i) != float:
        for v in i.split(","):
            categories_lst.append(v)